In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv("./pure-methods-pdfbox.csv")
df.shape

(1369, 6)

In [3]:
def extract_from_tags(tag, tags):
    search_string = tag + "=[\w]+"
    extracted_tag = re.findall(search_string, tags)[0]
    extracted_tag = re.findall("\=(.*)", extracted_tag)[0]
    if extracted_tag == "true":
        return True
    else:
        return False

In [4]:
cols = ["visibility", "parent-FQN", "method-name", "param-list", "return-type", 
        "local-variables", "conditionals", "multiple-statements", "loops", "parameters", "returns",
        "switches", "ifs", "static", "returns-primitives"]
final_df = pd.DataFrame(columns = cols)
for index, row in df.iterrows():
    final_df.loc[index, 'visibility'] = row['visibility']
    final_df.loc[index, 'parent-FQN'] = row['parent-FQN']
    final_df.loc[index, 'method-name'] = row['method-name']
    final_df.loc[index, 'param-list'] = row['param-list']
    final_df.loc[index, 'return-type'] = row['return-type']
    final_df.loc[index, 'local-variables'] = extract_from_tags("local_variables", str(row['tags']))
    final_df.loc[index, 'conditionals'] = extract_from_tags("conditionals", str(row['tags']))
    final_df.loc[index, 'multiple-statements'] = extract_from_tags("multiple_statements", str(row['tags']))
    final_df.loc[index, 'loops'] = extract_from_tags("loops", str(row['tags']))
    final_df.loc[index, 'parameters'] = extract_from_tags("parameters", str(row['tags']))
    final_df.loc[index, 'returns'] = extract_from_tags("returns", str(row['tags']))
    final_df.loc[index, 'switches'] = extract_from_tags("switches", str(row['tags']))
    final_df.loc[index, 'ifs'] = extract_from_tags("ifs", str(row['tags']))
    final_df.loc[index, 'static'] = extract_from_tags("static", str(row['tags']))
    final_df.loc[index, 'returns-primitives'] = extract_from_tags("returns_primitives", str(row['tags']))

final_df.head()

,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
0,public,org.apache.fontbox.cff.CIDKeyedType2CharString,getCID,[],int,False,False,False,False,False,True,False,False,False,True
1,public,org.apache.xmpbox.type.ArrayProperty,getArrayType,[],Cardinality,False,False,False,False,False,True,False,False,False,False
2,public,org.apache.fontbox.ttf.HeaderTable,getFontRevision,[],float,False,False,False,False,False,True,False,False,False,True
3,public,org.apache.fontbox.ttf.VerticalMetricsTable,getAdvanceHeight,[int],int,False,False,False,False,True,True,False,True,False,True
4,public,org.apache.pdfbox.text.TextPosition,getHeightDir,[],float,False,False,False,False,False,True,False,False,False,True


### Methods that return objects

In [5]:
(final_df["returns-primitives"] == False).sum()

754

### Methods that accept parameters

In [6]:
(final_df["parameters"] == True).sum()

151

### Methods with if statements

In [7]:
(final_df["ifs"] == True).sum()

37

### Methods wtih conditionals ?:

In [8]:
(final_df["conditionals"] == True).sum()

21

### Methods with local variables

In [9]:
(final_df["local-variables"] == True).sum()

40

### Methods with multiple statements

In [10]:
(final_df["multiple-statements"] == True).sum()

52

In [11]:
instrumentation_candidates_df = pd.DataFrame(columns = cols)
instrumentation_candidates_df = final_df[((final_df['multiple-statements'] == True) | 
                                          (final_df['ifs'] == True) |
                                          (final_df['conditionals'] == True) | 
                                          (final_df['parameters'] == True) | 
                                          (final_df['switches'] == True) |
                                          (final_df['loops'] == True) |
                                          (final_df['local-variables'] == True)) &
                                          (final_df['static'] == False) & 
                                          (final_df['visibility'] == "public")]
print(instrumentation_candidates_df.shape)
instrumentation_candidates_df.to_csv(r'./instrumentation-candidates-pdfbox.csv', index=False)
instrumentation_candidates_df.tail(60)

(52, 15)


,visibility,parent-FQN,method-name,param-list,return-type,local-variables,conditionals,multiple-statements,loops,parameters,returns,switches,ifs,static,returns-primitives
3,public,org.apache.fontbox.ttf.VerticalMetricsTable,getAdvanceHeight,[int],int,False,False,False,False,True,True,False,True,False,True
10,public,org.apache.fontbox.util.BoundingBox,contains,"[float, float]",boolean,False,False,False,False,True,True,False,False,False,True
32,public,org.apache.pdfbox.util.SmallMap,size,[],int,False,True,False,False,False,True,False,False,False,True
72,public,org.apache.fontbox.ttf.PostScriptTable,getName,[int],String,False,False,True,False,True,True,False,True,False,False
189,public,org.apache.fontbox.ttf.GlyfSimpleDescript,getEndPtOfContours,[int],int,False,False,False,False,True,True,False,False,False,True
197,public,org.apache.fontbox.ttf.GlyfSimpleDescript,getFlags,[int],byte,False,False,False,False,True,True,False,False,False,True
230,public,org.apache.fontbox.ttf.GlyfSimpleDescript,getYCoordinate,[int],short,False,False,False,False,True,True,False,False,False,True
279,public,org.apache.pdfbox.pdmodel.font.PDCIDFontType0,getCFFFont,[],CFFFont,False,False,False,False,False,True,False,True,False,False
456,public,org.apache.fontbox.cmap.CIDRange,unmap,[int],int,False,False,True,False,True,True,False,True,False,True
526,public,org.apache.pdfbox.util.Matrix,getValue,"[int, int]",float,False,False,False,False,True,True,False,False,False,True


In [12]:
sorted(instrumentation_candidates_df['parent-FQN'].unique())

['org.apache.fontbox.cff.CFFParser$Format0FDSelect',
 'org.apache.fontbox.cff.CFFParser$Format3FDSelect',
 'org.apache.fontbox.cff.CharStringCommand$Key',
 'org.apache.fontbox.cmap.CIDRange',
 'org.apache.fontbox.cmap.CodespaceRange',
 'org.apache.fontbox.ttf.GlyfSimpleDescript',
 'org.apache.fontbox.ttf.HorizontalMetricsTable',
 'org.apache.fontbox.ttf.KerningSubtable',
 'org.apache.fontbox.ttf.KerningSubtable$PairData0Format0',
 'org.apache.fontbox.ttf.OpenTypeScript$1',
 'org.apache.fontbox.ttf.PostScriptTable',
 'org.apache.fontbox.ttf.VerticalMetricsTable',
 'org.apache.fontbox.type1.Token',
 'org.apache.fontbox.util.BoundingBox',
 'org.apache.pdfbox.cos.COSBoolean',
 'org.apache.pdfbox.debugger.colorpane.DeviceNTableModel',
 'org.apache.pdfbox.debugger.colorpane.IndexedTableModel',
 'org.apache.pdfbox.debugger.ui.PageEntry',
 'org.apache.pdfbox.filter.ASCII85InputStream',
 'org.apache.pdfbox.pdfparser.PDFXRefStream',
 'org.apache.pdfbox.pdfparser.XrefTrailerResolver',
 'org.apach